In [8]:
from __future__ import print_function
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.image as mpimg
import cv2

import os, itertools, pickle

from util import *
from config import *
%load_ext autoreload
%aimport util
%aimport config
%autoreload 1

In [9]:
def get_chessboard(frames_dict):
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))

    w, h = 7, 9
    objp = np.zeros((w * h, 3), np.float32)
    objp[:, :2] = np.mgrid[:w, :h].T.reshape(-1, 2)
    
    point_dict = {} # maps image_path to (3d point in real world space, 2d point in image plane)
    for image_path, img in sorted(frames_dict.items()):
        gray = clahe.apply(cv2.cvtColor(img, cv2.COLOR_BGR2GRAY))

        ret, corners = cv2.findChessboardCorners(gray, (w, h))
        if ret:
            criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
            cv2.cornerSubPix(gray, corners, (11, 11), (-1, -1), criteria)
#             corner_img = cv2.drawChessboardCorners(img, (w, h), corners, True)
            print(image_path)
            point_dict[image_path] = (objp, corners)
        else:
            print('no', image_path)
            remove(image_path)
    return point_dict

def filter_nth(matrix, n):
    return np.array(matrix[::n])

In [13]:
# http://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_calib3d/py_calibration/py_calibration.html
# https://docs.opencv.org/2.4.2/modules/calib3d/doc/camera_calibration_and_3d_reconstruction.html

for camera_i in [1, 2, 3, 4, 5]:
    print('camera', camera_i)
    camera_dir = Calibration + '%s/' % camera_i
    image_dir = camera_dir + 'image/'
    video_dir = camera_dir + 'video/'
    mtx_path = camera_dir + 'camera_matrix.npy'
    dist_path = camera_dir + 'distortion_coefficients.npy'
    
    print('getting camera parameters')
    if os.path.exists(mtx_path) and os.path.exists(dist_path):
        mtx = np.load(mtx_path)
        dist_coef = np.load(dist_path)
    else:
        object_points, image_points = [], []
        for video_name, video_path in list_dir(video_dir, 'mp4', return_name=True):
            point_record = video_path + '.records'
            print('video', video_path)
            if os.path.exists(point_record):
                print('loading saved points')
                point_dict = load_pickle(point_record)
            else:
                print('reading')
                video_name_dir = video_dir + video_name + '/'
                if not os.path.exists(video_name_dir):
                    video_to_frames(video_path, video_name_dir, nth=5)
                video_dict = { image_path : cv2.imread(image_path) for image_path in list_dir(video_name_dir, 'jpg')}
                print('finding points on chessboard for %s frames' % len(video_dict))
                point_dict = get_chessboard(video_dict)
                point_dict = { int(path.split('_')[-1].rstrip('.jpg')) : (path, points) for path, points in point_dict.items() }
                
                filtered_pt_dict = {}
                min_interval = 5
                prev_key = -min_interval
                for key, (path, points) in sorted(point_dict.items()):
                    if key - prev_key >= min_interval:
                        filtered_pt_dict[key] = points
                        prev_key = key
                    else:
                        remove(path)
                point_dict = filtered_pt_dict
                save_pickle(point_dict, point_record)
            for obj_pt, img_pt in point_dict.values():
                object_points.append(obj_pt)
                image_points.append(img_pt)
        if len(object_points) < 25:
            print('not enough matches for camera. Had %s' % len(object_points))
            continue
        else:
            print('found %s matches for camera' % len(object_points))
        
        ret, mtx, dist_coef, rvecs, tvecs = cv2.calibrateCamera(object_points, image_points, (Video_width, Video_height), None, None, flags=cv2.CALIB_RATIONAL_MODEL)
        
        np.save(mtx_path, mtx)
        np.save(dist_path, dist_coef)
        
        total_error = 0
        for i in range(len(object_points)):
            image_points2, _ = cv2.projectPoints(object_points[i], rvecs[i], tvecs[i], mtx, dist_coef)
            error = cv2.norm(image_points[i], image_points2, cv2.NORM_L2) / len(image_points2)
            total_error += error
        print('mean error:', total_error / len(object_points))
        
    print('getting new camera matrix')
    newcameramtx, roi = cv2.getOptimalNewCameraMatrix(mtx, dist_coef, (Video_width, Video_height), 0)
    if not any(roi):
        print('zeros for roi')
        continue
    print('generated new camera matrix')
    output_dir = make_dir(camera_dir + 'output/')
    for v_name, v_path in list_dir(camera_dir + 'test/', 'mp4', return_name=True):
        print('testing', v_path)
        shell('python undistort.py %s %s %s' % (v_path, camera_dir, output_dir + v_name + '.mp4'))
print('done')

camera 1
getting camera parameters
video /mnt/c/Users/zhong/Desktop/bdd/calibration/1/video/A0010072.MP4
loading saved points
video /mnt/c/Users/zhong/Desktop/bdd/calibration/1/video/GOPR0080.MP4
loading saved points
video /mnt/c/Users/zhong/Desktop/bdd/calibration/1/video/GOPR0084.MP4
loading saved points
found 74 matches for camera
mean error: 0.244656140777
getting new camera matrix
generated new camera matrix
testing /mnt/c/Users/zhong/Desktop/bdd/calibration/1/test/test.mp4
camera 2
getting camera parameters
video /mnt/c/Users/zhong/Desktop/bdd/calibration/2/video/GOPR0093.MP4
loading saved points
video /mnt/c/Users/zhong/Desktop/bdd/calibration/2/video/GOPR0104.MP4
loading saved points
found 83 matches for camera
mean error: 0.243003379587
getting new camera matrix
generated new camera matrix
testing /mnt/c/Users/zhong/Desktop/bdd/calibration/2/test/test.mp4
camera 3
getting camera parameters
video /mnt/c/Users/zhong/Desktop/bdd/calibration/3/video/GOPR0059.MP4
reading
finding po

no /mnt/c/Users/zhong/Desktop/bdd/calibration/3/video/GOPR0059/GOPR0059_0090.jpg
no /mnt/c/Users/zhong/Desktop/bdd/calibration/3/video/GOPR0059/GOPR0059_0091.jpg
no /mnt/c/Users/zhong/Desktop/bdd/calibration/3/video/GOPR0059/GOPR0059_0092.jpg
no /mnt/c/Users/zhong/Desktop/bdd/calibration/3/video/GOPR0059/GOPR0059_0093.jpg
no /mnt/c/Users/zhong/Desktop/bdd/calibration/3/video/GOPR0059/GOPR0059_0094.jpg
no /mnt/c/Users/zhong/Desktop/bdd/calibration/3/video/GOPR0059/GOPR0059_0095.jpg
no /mnt/c/Users/zhong/Desktop/bdd/calibration/3/video/GOPR0059/GOPR0059_0096.jpg
no /mnt/c/Users/zhong/Desktop/bdd/calibration/3/video/GOPR0059/GOPR0059_0097.jpg
no /mnt/c/Users/zhong/Desktop/bdd/calibration/3/video/GOPR0059/GOPR0059_0098.jpg
no /mnt/c/Users/zhong/Desktop/bdd/calibration/3/video/GOPR0059/GOPR0059_0099.jpg
no /mnt/c/Users/zhong/Desktop/bdd/calibration/3/video/GOPR0059/GOPR0059_0100.jpg
/mnt/c/Users/zhong/Desktop/bdd/calibration/3/video/GOPR0059/GOPR0059_0101.jpg
/mnt/c/Users/zhong/Desktop/bdd/

/mnt/c/Users/zhong/Desktop/bdd/calibration/3/video/GOPR0059/GOPR0059_0194.jpg
no /mnt/c/Users/zhong/Desktop/bdd/calibration/3/video/GOPR0059/GOPR0059_0195.jpg
/mnt/c/Users/zhong/Desktop/bdd/calibration/3/video/GOPR0059/GOPR0059_0196.jpg
/mnt/c/Users/zhong/Desktop/bdd/calibration/3/video/GOPR0059/GOPR0059_0197.jpg
/mnt/c/Users/zhong/Desktop/bdd/calibration/3/video/GOPR0059/GOPR0059_0198.jpg
/mnt/c/Users/zhong/Desktop/bdd/calibration/3/video/GOPR0059/GOPR0059_0199.jpg
no /mnt/c/Users/zhong/Desktop/bdd/calibration/3/video/GOPR0059/GOPR0059_0200.jpg
no /mnt/c/Users/zhong/Desktop/bdd/calibration/3/video/GOPR0059/GOPR0059_0201.jpg
/mnt/c/Users/zhong/Desktop/bdd/calibration/3/video/GOPR0059/GOPR0059_0202.jpg
/mnt/c/Users/zhong/Desktop/bdd/calibration/3/video/GOPR0059/GOPR0059_0203.jpg
/mnt/c/Users/zhong/Desktop/bdd/calibration/3/video/GOPR0059/GOPR0059_0204.jpg
/mnt/c/Users/zhong/Desktop/bdd/calibration/3/video/GOPR0059/GOPR0059_0205.jpg
/mnt/c/Users/zhong/Desktop/bdd/calibration/3/video/GOPR

/mnt/c/Users/zhong/Desktop/bdd/calibration/3/video/GOPR0059/GOPR0059_0299.jpg
/mnt/c/Users/zhong/Desktop/bdd/calibration/3/video/GOPR0059/GOPR0059_0300.jpg
/mnt/c/Users/zhong/Desktop/bdd/calibration/3/video/GOPR0059/GOPR0059_0301.jpg
no /mnt/c/Users/zhong/Desktop/bdd/calibration/3/video/GOPR0059/GOPR0059_0302.jpg
/mnt/c/Users/zhong/Desktop/bdd/calibration/3/video/GOPR0059/GOPR0059_0303.jpg
/mnt/c/Users/zhong/Desktop/bdd/calibration/3/video/GOPR0059/GOPR0059_0304.jpg
/mnt/c/Users/zhong/Desktop/bdd/calibration/3/video/GOPR0059/GOPR0059_0305.jpg
/mnt/c/Users/zhong/Desktop/bdd/calibration/3/video/GOPR0059/GOPR0059_0306.jpg
/mnt/c/Users/zhong/Desktop/bdd/calibration/3/video/GOPR0059/GOPR0059_0307.jpg
/mnt/c/Users/zhong/Desktop/bdd/calibration/3/video/GOPR0059/GOPR0059_0308.jpg
/mnt/c/Users/zhong/Desktop/bdd/calibration/3/video/GOPR0059/GOPR0059_0309.jpg
/mnt/c/Users/zhong/Desktop/bdd/calibration/3/video/GOPR0059/GOPR0059_0310.jpg
no /mnt/c/Users/zhong/Desktop/bdd/calibration/3/video/GOPR005

/mnt/c/Users/zhong/Desktop/bdd/calibration/3/video/GOPR0059/GOPR0059_0404.jpg
/mnt/c/Users/zhong/Desktop/bdd/calibration/3/video/GOPR0059/GOPR0059_0405.jpg
/mnt/c/Users/zhong/Desktop/bdd/calibration/3/video/GOPR0059/GOPR0059_0406.jpg
/mnt/c/Users/zhong/Desktop/bdd/calibration/3/video/GOPR0059/GOPR0059_0407.jpg
/mnt/c/Users/zhong/Desktop/bdd/calibration/3/video/GOPR0059/GOPR0059_0408.jpg
/mnt/c/Users/zhong/Desktop/bdd/calibration/3/video/GOPR0059/GOPR0059_0409.jpg
no /mnt/c/Users/zhong/Desktop/bdd/calibration/3/video/GOPR0059/GOPR0059_0410.jpg
/mnt/c/Users/zhong/Desktop/bdd/calibration/3/video/GOPR0059/GOPR0059_0411.jpg
/mnt/c/Users/zhong/Desktop/bdd/calibration/3/video/GOPR0059/GOPR0059_0412.jpg
/mnt/c/Users/zhong/Desktop/bdd/calibration/3/video/GOPR0059/GOPR0059_0413.jpg
/mnt/c/Users/zhong/Desktop/bdd/calibration/3/video/GOPR0059/GOPR0059_0414.jpg
/mnt/c/Users/zhong/Desktop/bdd/calibration/3/video/GOPR0059/GOPR0059_0415.jpg
/mnt/c/Users/zhong/Desktop/bdd/calibration/3/video/GOPR0059/G

...

/mnt/c/Users/zhong/Desktop/bdd/calibration/4/video/GOPR0057/GOPR0057_0340.jpg
/mnt/c/Users/zhong/Desktop/bdd/calibration/4/video/GOPR0057/GOPR0057_0341.jpg
/mnt/c/Users/zhong/Desktop/bdd/calibration/4/video/GOPR0057/GOPR0057_0342.jpg
no /mnt/c/Users/zhong/Desktop/bdd/calibration/4/video/GOPR0057/GOPR0057_0343.jpg
no /mnt/c/Users/zhong/Desktop/bdd/calibration/4/video/GOPR0057/GOPR0057_0344.jpg
/mnt/c/Users/zhong/Desktop/bdd/calibration/4/video/GOPR0057/GOPR0057_0345.jpg
/mnt/c/Users/zhong/Desktop/bdd/calibration/4/video/GOPR0057/GOPR0057_0346.jpg
/mnt/c/Users/zhong/Desktop/bdd/calibration/4/video/GOPR0057/GOPR0057_0347.jpg
/mnt/c/Users/zhong/Desktop/bdd/calibration/4/video/GOPR0057/GOPR0057_0348.jpg
/mnt/c/Users/zhong/Desktop/bdd/calibration/4/video/GOPR0057/GOPR0057_0349.jpg
/mnt/c/Users/zhong/Desktop/bdd/calibration/4/video/GOPR0057/GOPR0057_0350.jpg
no /mnt/c/Users/zhong/Desktop/bdd/calibration/4/video/GOPR0057/GOPR0057_0351.jpg
no /mnt/c/Users/zhong/Desktop/bdd/calibration/4/video/G

/mnt/c/Users/zhong/Desktop/bdd/calibration/4/video/GOPR0057/GOPR0057_0445.jpg
/mnt/c/Users/zhong/Desktop/bdd/calibration/4/video/GOPR0057/GOPR0057_0446.jpg
/mnt/c/Users/zhong/Desktop/bdd/calibration/4/video/GOPR0057/GOPR0057_0447.jpg
/mnt/c/Users/zhong/Desktop/bdd/calibration/4/video/GOPR0057/GOPR0057_0448.jpg
/mnt/c/Users/zhong/Desktop/bdd/calibration/4/video/GOPR0057/GOPR0057_0449.jpg
/mnt/c/Users/zhong/Desktop/bdd/calibration/4/video/GOPR0057/GOPR0057_0450.jpg
/mnt/c/Users/zhong/Desktop/bdd/calibration/4/video/GOPR0057/GOPR0057_0451.jpg
/mnt/c/Users/zhong/Desktop/bdd/calibration/4/video/GOPR0057/GOPR0057_0452.jpg
/mnt/c/Users/zhong/Desktop/bdd/calibration/4/video/GOPR0057/GOPR0057_0453.jpg
/mnt/c/Users/zhong/Desktop/bdd/calibration/4/video/GOPR0057/GOPR0057_0454.jpg
no /mnt/c/Users/zhong/Desktop/bdd/calibration/4/video/GOPR0057/GOPR0057_0455.jpg
/mnt/c/Users/zhong/Desktop/bdd/calibration/4/video/GOPR0057/GOPR0057_0456.jpg
/mnt/c/Users/zhong/Desktop/bdd/calibration/4/video/GOPR0057/G

/mnt/c/Users/zhong/Desktop/bdd/calibration/4/video/GOPR0057/GOPR0057_0550.jpg
/mnt/c/Users/zhong/Desktop/bdd/calibration/4/video/GOPR0057/GOPR0057_0551.jpg
no /mnt/c/Users/zhong/Desktop/bdd/calibration/4/video/GOPR0057/GOPR0057_0552.jpg
/mnt/c/Users/zhong/Desktop/bdd/calibration/4/video/GOPR0057/GOPR0057_0553.jpg
/mnt/c/Users/zhong/Desktop/bdd/calibration/4/video/GOPR0057/GOPR0057_0554.jpg
/mnt/c/Users/zhong/Desktop/bdd/calibration/4/video/GOPR0057/GOPR0057_0555.jpg
/mnt/c/Users/zhong/Desktop/bdd/calibration/4/video/GOPR0057/GOPR0057_0556.jpg
/mnt/c/Users/zhong/Desktop/bdd/calibration/4/video/GOPR0057/GOPR0057_0557.jpg
/mnt/c/Users/zhong/Desktop/bdd/calibration/4/video/GOPR0057/GOPR0057_0558.jpg
/mnt/c/Users/zhong/Desktop/bdd/calibration/4/video/GOPR0057/GOPR0057_0559.jpg
no /mnt/c/Users/zhong/Desktop/bdd/calibration/4/video/GOPR0057/GOPR0057_0560.jpg
/mnt/c/Users/zhong/Desktop/bdd/calibration/4/video/GOPR0057/GOPR0057_0561.jpg
/mnt/c/Users/zhong/Desktop/bdd/calibration/4/video/GOPR005

/mnt/c/Users/zhong/Desktop/bdd/calibration/4/video/GOPR0057/GOPR0057_0655.jpg
/mnt/c/Users/zhong/Desktop/bdd/calibration/4/video/GOPR0057/GOPR0057_0656.jpg
/mnt/c/Users/zhong/Desktop/bdd/calibration/4/video/GOPR0057/GOPR0057_0657.jpg
/mnt/c/Users/zhong/Desktop/bdd/calibration/4/video/GOPR0057/GOPR0057_0658.jpg
/mnt/c/Users/zhong/Desktop/bdd/calibration/4/video/GOPR0057/GOPR0057_0659.jpg
no /mnt/c/Users/zhong/Desktop/bdd/calibration/4/video/GOPR0057/GOPR0057_0660.jpg
/mnt/c/Users/zhong/Desktop/bdd/calibration/4/video/GOPR0057/GOPR0057_0661.jpg
/mnt/c/Users/zhong/Desktop/bdd/calibration/4/video/GOPR0057/GOPR0057_0662.jpg
/mnt/c/Users/zhong/Desktop/bdd/calibration/4/video/GOPR0057/GOPR0057_0663.jpg
/mnt/c/Users/zhong/Desktop/bdd/calibration/4/video/GOPR0057/GOPR0057_0664.jpg
/mnt/c/Users/zhong/Desktop/bdd/calibration/4/video/GOPR0057/GOPR0057_0665.jpg
/mnt/c/Users/zhong/Desktop/bdd/calibration/4/video/GOPR0057/GOPR0057_0666.jpg
/mnt/c/Users/zhong/Desktop/bdd/calibration/4/video/GOPR0057/G

no /mnt/c/Users/zhong/Desktop/bdd/calibration/4/video/GOPR0057/GOPR0057_0759.jpg
no /mnt/c/Users/zhong/Desktop/bdd/calibration/4/video/GOPR0057/GOPR0057_0760.jpg
no /mnt/c/Users/zhong/Desktop/bdd/calibration/4/video/GOPR0057/GOPR0057_0761.jpg
no /mnt/c/Users/zhong/Desktop/bdd/calibration/4/video/GOPR0057/GOPR0057_0762.jpg
no /mnt/c/Users/zhong/Desktop/bdd/calibration/4/video/GOPR0057/GOPR0057_0763.jpg
no /mnt/c/Users/zhong/Desktop/bdd/calibration/4/video/GOPR0057/GOPR0057_0764.jpg
no /mnt/c/Users/zhong/Desktop/bdd/calibration/4/video/GOPR0057/GOPR0057_0765.jpg
no /mnt/c/Users/zhong/Desktop/bdd/calibration/4/video/GOPR0057/GOPR0057_0766.jpg
no /mnt/c/Users/zhong/Desktop/bdd/calibration/4/video/GOPR0057/GOPR0057_0767.jpg
no /mnt/c/Users/zhong/Desktop/bdd/calibration/4/video/GOPR0057/GOPR0057_0768.jpg
no /mnt/c/Users/zhong/Desktop/bdd/calibration/4/video/GOPR0057/GOPR0057_0769.jpg
no /mnt/c/Users/zhong/Desktop/bdd/calibration/4/video/GOPR0057/GOPR0057_0770.jpg
/mnt/c/Users/zhong/Desktop/b